In [3]:
import torch
import torch.nn as nn

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [8]:
#def embedding_gen(sentence):
sentence = "I need ya ya"
word_idx = {word:i for i, word in enumerate(set(sentence.split()))}
print(word_idx)

{'need': 0, 'ya': 1, 'I': 2}


In [9]:
token_to_id = {'what': 0,
                'is': 1,
                'statquest': 2,
                'awesome': 3,
                '<EOS>': 4}

id_to_token = dict(map(reversed, token_to_id.items()))

{0: 'what', 1: 'is', 2: 'statquest', 3: 'awesome', 4: '<EOS>'}